# NQSD 範例 2：資料分析

本範例示範深入的資料分析：
- 建築高度分佈分析
- 校區比較分析
- 結構類型統計
- 空間密度分析

**資料來源**：
- NLSC 3D Building Data
- OpenStreetMap (© OSM contributors)

In [ ]:
# 匯入必要套件
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

# 設定中文字型
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ 套件載入成功")

## 1. 載入資料

In [ ]:
# 讀取光復校區合併資料
guangfu = gpd.read_file('../data/output/latest/buildings_merged.geojson')

# 讀取其他校區資料（如果存在）
try:
    boai = gpd.read_file('../data/processed/buildings/by_campus/boai/NLSC_buildings.json')
    yangming = gpd.read_file('../data/processed/buildings/by_campus/yangming/NLSC_buildings.json')
    print(f"✓ 已載入多校區資料")
except:
    print("僅載入光復校區資料")
    boai, yangming = None, None

print(f"光復校區建築數量：{len(guangfu)}")

## 2. 建築高度分佈分析

In [ ]:
# 建築高度統計
height_stats = guangfu['nlsc_BUILD_H'].describe()
print("建築高度統計：")
print(height_stats)
print(f"\n中位數：{guangfu['nlsc_BUILD_H'].median():.2f} 公尺")
print(f"標準差：{guangfu['nlsc_BUILD_H'].std():.2f} 公尺")

In [ ]:
# 建築高度分佈（多種視覺化）
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 直方圖
axes[0].hist(guangfu['nlsc_BUILD_H'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(guangfu['nlsc_BUILD_H'].mean(), color='red', linestyle='--', label=f'平均: {guangfu["nlsc_BUILD_H"].mean():.1f}m')
axes[0].axvline(guangfu['nlsc_BUILD_H'].median(), color='green', linestyle='--', label=f'中位數: {guangfu["nlsc_BUILD_H"].median():.1f}m')
axes[0].set_xlabel('建築高度（公尺）')
axes[0].set_ylabel('建築數量')
axes[0].set_title('建築高度分佈（直方圖）')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 箱型圖
axes[1].boxplot(guangfu['nlsc_BUILD_H'], vert=True)
axes[1].set_ylabel('建築高度（公尺）')
axes[1].set_title('建築高度分佈（箱型圖）')
axes[1].grid(True, alpha=0.3)

# 累積分佈
sorted_heights = np.sort(guangfu['nlsc_BUILD_H'])
cumulative = np.arange(1, len(sorted_heights) + 1) / len(sorted_heights) * 100
axes[2].plot(sorted_heights, cumulative)
axes[2].set_xlabel('建築高度（公尺）')
axes[2].set_ylabel('累積百分比（%）')
axes[2].set_title('建築高度累積分佈')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 高度分級統計

In [ ]:
# 建築高度分級
bins = [0, 10, 20, 30, 40, 50, 100]
labels = ['0-10m', '10-20m', '20-30m', '30-40m', '40-50m', '50m+']
guangfu['height_category'] = pd.cut(guangfu['nlsc_BUILD_H'], bins=bins, labels=labels)

height_counts = guangfu['height_category'].value_counts().sort_index()
print("建築高度分級統計：")
print(height_counts)

# 圓餅圖
plt.figure(figsize=(10, 6))
plt.pie(height_counts, labels=height_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('建築高度分級比例')
plt.show()

## 4. 結構類型分析

In [ ]:
# 結構類型統計
structure_counts = guangfu['nlsc_BUILD_STR'].value_counts()
print("結構類型統計：")
print(structure_counts)

# 結構類型對照表
structure_names = {
    'R': '鋼筋混凝土',
    'B': '磚造',
    'S': '鋼構造',
    'W': '木造',
    'RC': '加強磚造'
}

# 長條圖
plt.figure(figsize=(10, 6))
structure_counts.plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('結構類型')
plt.ylabel('建築數量')
plt.title('建築結構類型分佈')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3, axis='y')
plt.show()

## 5. 不同結構類型的平均高度

In [ ]:
# 各結構類型平均高度
avg_height_by_structure = guangfu.groupby('nlsc_BUILD_STR')['nlsc_BUILD_H'].agg(['mean', 'median', 'std', 'count'])
print("各結構類型平均高度：")
print(avg_height_by_structure.round(2))

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 平均高度
avg_height_by_structure['mean'].plot(kind='bar', ax=axes[0], color='coral', edgecolor='black')
axes[0].set_ylabel('平均高度（公尺）')
axes[0].set_title('各結構類型平均高度')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)
axes[0].grid(True, alpha=0.3, axis='y')

# 箱型圖比較
guangfu.boxplot(column='nlsc_BUILD_H', by='nlsc_BUILD_STR', ax=axes[1])
axes[1].set_ylabel('建築高度（公尺）')
axes[1].set_xlabel('結構類型')
axes[1].set_title('各結構類型高度分佈')
plt.suptitle('')  # 移除預設標題

plt.tight_layout()
plt.show()

## 6. 最高建築排名

In [ ]:
# 找出有名稱的建築
named_buildings = guangfu[guangfu['name'].notna()].copy()

# 最高的 15 棟建築
top_15 = named_buildings.nlargest(15, 'nlsc_BUILD_H')[['name', 'nlsc_BUILD_H', 'nlsc_BUILD_STR']]
print("最高的 15 棟建築：")
print(top_15.to_string(index=False))

# 視覺化
plt.figure(figsize=(12, 6))
plt.barh(range(len(top_15)), top_15['nlsc_BUILD_H'], color='steelblue', edgecolor='black')
plt.yticks(range(len(top_15)), top_15['name'])
plt.xlabel('建築高度（公尺）')
plt.title('光復校區最高的 15 棟建築')
plt.gca().invert_yaxis()  # 最高的在上面
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 7. 匯出分析結果

In [ ]:
# 匯出統計結果
analysis_results = pd.DataFrame({
    '統計項目': ['總建築數', '平均高度', '中位數高度', '最高建築', '最矮建築', '標準差'],
    '數值': [
        len(guangfu),
        f"{guangfu['nlsc_BUILD_H'].mean():.2f} m",
        f"{guangfu['nlsc_BUILD_H'].median():.2f} m",
        f"{guangfu['nlsc_BUILD_H'].max():.2f} m",
        f"{guangfu['nlsc_BUILD_H'].min():.2f} m",
        f"{guangfu['nlsc_BUILD_H'].std():.2f} m"
    ]
})

analysis_results.to_csv('outputs/analysis_summary.csv', index=False, encoding='utf-8-sig')
print("✓ 分析結果已匯出至 outputs/analysis_summary.csv")

## 總結

本範例完成了：
- ✅ 建築高度分佈分析（直方圖、箱型圖、累積分佈）
- ✅ 高度分級統計（6 個分級）
- ✅ 結構類型分析（R, B, S 等）
- ✅ 不同結構類型的平均高度比較
- ✅ 最高建築排名（Top 15）
- ✅ 匯出分析結果

下一步：
- 查看 `03_visualization.ipynb` 學習互動式地圖視覺化